<a href="https://colab.research.google.com/github/Nichalaks/GG_Collabs_ns/blob/main/6720422019_Redesign_Interface_Premier_League.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Premier League — Inline Dashboard (Plotly Go, no HTML file)
# - Filters: FILTER_TEAMS (list[str] or None), POINTS_RANGE (min_pts, max_pts)
# - Output: Standings Table (top) + 6 charts, all rendered inline with fig.show()

import pandas as pd
import plotly.graph_objects as go

# =========================
# 0) FILTERS (แก้ค่านี้ได้)
# =========================
# ตัวอย่าง:
# FILTER_TEAMS = ["Arsenal", "Manchester City", "Liverpool"]
# POINTS_RANGE = (10, 20)
FILTER_TEAMS = None          # None = ทุกทีม | หรือ list ชื่อทีม
POINTS_RANGE = (0, 1000)     # กำหนดช่วงคะแนนที่ต้องการ

# =========================
# 1) DATA (จากสกรีนช็อต)
# =========================
rows = [
    [1,  "Arsenal",            8, 6, 1, 1, 18,  6, "+12", 19, 2.38, 13.6,  6.7,  "+6.9",  "+1.11", "🟩🟨🟩🟩🟩", "60,155", "Viktor Gyökeres - 3",                        "David Raya",                 ""],
    [2,  "Sunderland",         9, 5, 2, 2, 11,  7,  "+4", 17, 1.89,  8.5, 10.4,  "-2.0",  "-0.22", "🟨🟩🟥🟩🟩", "46,335", "Wilson Isidor - 4",                           "Robin Roefs",                ""],
    [3,  "Manchester City",    8, 5, 1, 2, 17,  6, "+11", 16, 2.00, 14.4,  8.2,  "+6.3",  "+0.78", "🟩🟥🟩🟩🟥", "52,311", "Erling Haaland - 11",                       "Gianluigi Donnarumma",       ""],
    [4,  "Manchester Utd",     9, 5, 1, 3, 15, 13,  "+2", 16, 1.78, 16.1, 13.6,  "+2.5",  "+0.28", "🟥🟩🟩🟩🟩", "73,965", "Bryan Mbeumo - 4",                         "Altay Bayındır",             ""],
    [5,  "Bournemouth",        8, 4, 3, 1, 11,  8,  "+3", 15, 1.88, 10.4, 10.7,  "-0.3",  "-0.04", "🟨🟨🟩🟨🟩", "11,154", "Antoine Semenyo - 6",                       "Đorđe Petrović",             ""],
    [6,  "Liverpool",          9, 5, 0, 4, 16, 12,  "+4", 15, 1.67, 15.7, 11.4,  "+4.2",  "+0.47", "🟩🟥🟥🟥🟥", "60,362", "Mohamed Salah, Cody Gakpo — 3",            "Alisson",                    ""],
    [7,  "Tottenham",          8, 4, 2, 2, 14,  7,  "+7", 14, 1.75,  8.2,  9.0,  "-0.8",  "-0.10", "🟥🟨🟥🟩🟥", "61,039", "Richarlison — 3",                           "Guglielmo Vicario",          ""],
    [8,  "Chelsea",            9, 4, 2, 3, 12, 11,  "+1", 14, 1.56, 13.5, 13.0,  "+0.5",  "+0.06", "🟩🟥🟥🟥🟥", "39,640", "Enzo Fernández, Moisés Caicedo — 3",       "Robert Sánchez",             ""],
    [9,  "Crystal Palace",     9, 3, 4, 2, 11,  7,  "+4", 13, 1.63, 17.1, 10.8,  "+6.3",  "+0.79", "🟨🟩🟩🟨🟥", "25,068", "Jean-Philippe Mateta — 5",                 "Dean Henderson",             ""],
    [10, "Brentford",          9, 4, 1, 4, 14, 10,  "+4", 13, 1.44, 12.9,  9.4,  "+3.6",  "+0.54", "🟩🟥🟩🟩🟩", "17,014", "Thiago — 6",                                "Caoimhín Kelleher",          ""],
    [11, "Newcastle Utd",      9, 3, 3, 3, 10, 11,  "-1", 12, 1.33, 15.9, 13.0,  "+2.9",  "+0.41", "🟨🟩🟥🟩🟩", "52,183", "Nick Woltmerlade — 4",                      "Nick Pope",                  ""],
    [12, "Aston Villa",        8, 3, 3, 2,  9,  9,   "0", 12, 1.50, 11.0,  6.6,  "+4.3",  "+0.54", "🟨🟨🟩🟩🟩", "41,055", "Emi Buendía, Donyell Malen — 2",           "Emiliano Martínez",          ""],
    [13, "Brighton",           9, 3, 3, 3, 12, 11,  "+1", 12, 1.33, 13.1, 12.0,  "+1.1",  "+0.17", "🟨🟨🟥🟨🟥", "31,518", "Danny Welbeck — 5",                        "Bart Verbruggen",            ""],
    [14, "Everton",            9, 3, 2, 4,  9, 11,  "-2", 11, 1.22, 10.8, 11.9,  "-1.1",  "-0.13", "🟨🟥🟨🟩🟩", "51,883", "Iliman Ndiaye — 3",                         "Jordan Pickford",            ""],
    [15, "Leeds United",       9, 2, 4, 3, 11, 12,  "-1", 10, 1.11, 12.3, 10.9,  "+1.4",  "+0.28", "🟨🟩🟨🟩🟩", "36,722", "Joe Rodon, Noah Okafor — 2",               "Karl Darlow",                ""],
    [16, "Fulham",             9, 2, 2, 5,  9, 15,  "-6",  8, 0.89,  9.0, 12.7,  "-3.6",  "-0.42", "🟩🟩🟥🟥🟥", "27,172", "Raúl Jiménez, Alex Iwobi — 2",              "Bernd Leno",                 ""],
    [17, "Burnley",            9, 2, 1, 6,  8, 15,  "-7",  7, 0.78,  9.1, 15.4,  "-6.3",  "-1.01", "🟨🟥🟨🟥🟥", "21,521", "Jaidon Anthony — 4",                        "Martin Dúbravka",            ""],
    [18, "Nottingham Forest",  8, 1, 2, 5,  5, 10,  "-5",  5, 0.63, 10.0, 17.9,  "-7.9",  "-0.67", "🟥🟥🟨🟥🟥", "30,221", "Jhon Jader Durán — 2",                      "Matija Šarkić",              ""],
    [19, "West Ham",           8, 1, 1, 6,  6, 17, "-11",  4, 0.50,  9.6, 16.9,  "-7.2",  "-0.90", "🟥🟨🟥🟥🟥", "20,922", "Jarrod Bowen — 3",                          "Alphonse Areola",            ""],
    [20, "Wolves",             8, 0, 2, 6,  5, 16, "-11",  2, 0.25,  6.9, 16.0,  "-9.1",  "-0.39", "🟥🟨🟥🟥🟨", "30,510", "Hwang Hee-chan, Santiago Bueno — 1",        "Sam Johnstone, José Sá",     ""],
]

columns = [
    "Rk","Squad","MP","W","D","L","GF","GA","GD","Pts","Pts/MP",
    "xG","xGA","xGD","xGD/90","Last5","Attendance","Top Team Scorer","Goalkeeper","Notes"
]
df = pd.DataFrame(rows, columns=columns)

# =========================
# 2) CLEAN / FEATURES
# =========================
def parse_signed(x):
    if isinstance(x, (int, float)):
        return x
    s = str(x).replace(",", "").strip()
    try:
        return int(s.replace("+",""))
    except ValueError:
        try:
            return float(s.replace("+",""))
        except ValueError:
            return None

def parse_int_commas(x):
    return int(str(x).replace(",", ""))

df["GD_num"]      = df["GD"].apply(parse_signed)
df["xGD_num"]     = df["xGD"].apply(parse_signed)
df["xGD90_num"]   = df["xGD/90"].apply(parse_signed)
df["AttendanceN"] = df["Attendance"].apply(parse_int_commas)

def last5_counts(s):
    return pd.Series({"W5": s.count("🟩"), "D5": s.count("🟨"), "L5": s.count("🟥")})
df = pd.concat([df, df["Last5"].apply(last5_counts)], axis=1)

# =========================
# 3) APPLY FILTERS
# =========================
f_min, f_max = POINTS_RANGE
mask_pts = (df["Pts"] >= f_min) & (df["Pts"] <= f_max)
mask_team = df["Squad"].isin(FILTER_TEAMS) if FILTER_TEAMS else True
df_f = df[mask_pts & mask_team].copy()

# ถ้าไม่มีทีมตรง filter ให้แสดงข้อความเตือนด้วยตารางเปล่าๆ
if df_f.empty:
    print("⚠️ No data matched the current filters. Adjust FILTER_TEAMS / POINTS_RANGE.")
    df_f = df.iloc[0:0].copy()

# =========================
# 4) STANDINGS TABLE (บนสุด)
# =========================
stand_cols = ["Rk","Squad","MP","W","D","L","GF","GA","GD","Pts","Pts/MP","xG","xGA","xGD","xGD/90","Last5","Attendance","Top Team Scorer","Goalkeeper","Notes"]
df_table = df_f.sort_values(["Pts","GD_num","GF"], ascending=[False, False, False])[stand_cols]

fig_table = go.Figure(
    data=[
        go.Table(
            header=dict(
                values=[f"<b>{c}</b>" for c in df_table.columns],
                fill_color="#f2f2f2",
                align=["right","left"] + ["center"]*(len(df_table.columns)-2),
                height=32
            ),
            cells=dict(
                values=[df_table[c] for c in df_table.columns],
                align=["right","left"] + ["center"]*(len(df_table.columns)-2),
                height=26,
                fill_color=[
                    ["#ffffff" if i%2==0 else "#fbfbfb" for i in range(len(df_table))]
                    for _ in df_table.columns
                ],
            )
        )
    ]
)
fig_table.update_layout(
    title=f"Standings (Filtered) — Teams: {('All' if not FILTER_TEAMS else ', '.join(FILTER_TEAMS))} | Points: {f_min}–{f_max}",
    margin=dict(l=8, r=8, t=50, b=8),
    height= (80 + 26*max(1, len(df_table)))  # ปรับความสูงตามจำนวนแถว
)
fig_table.show()

# =========================
# 5) CHARTS (ใช้ df_f หลังฟิลเตอร์)
# =========================

# 5.1 Points (sorted)
df_pts = df_f.sort_values("Pts", ascending=False)
fig1 = go.Figure()
fig1.add_trace(go.Bar(x=df_pts["Squad"], y=df_pts["Pts"], text=df_pts["Pts"], textposition="outside"))
fig1.update_layout(title="Points (sorted)", xaxis_title="", yaxis_title="Points", height=430, margin=dict(l=20,r=20,t=60,b=40))
fig1.show()

# 5.2 Goal Difference (GD)
df_gd = df_f.sort_values("GD_num", ascending=False)
fig2 = go.Figure()
fig2.add_trace(go.Bar(x=df_gd["Squad"], y=df_gd["GD_num"], text=df_gd["GD"], textposition="outside"))
fig2.update_layout(title="Goal Difference (GD)", xaxis_title="", yaxis_title="GD", height=430, margin=dict(l=20,r=20,t=60,b=40))
fig2.show()

# 5.3 xGD vs Points
fig3 = go.Figure()
fig3.add_trace(go.Scatter(x=df_f["xGD_num"], y=df_f["Pts"], mode="markers+text", text=df_f["Squad"], textposition="top center"))
fig3.update_layout(title="xGD vs Points", xaxis_title="xGD", yaxis_title="Points", height=460, margin=dict(l=20,r=20,t=60,b=40))
fig3.show()

# 5.4 Attendance (Top 10 ของชุดที่ฟิลเตอร์แล้ว)
df_att = df_f.sort_values("AttendanceN", ascending=False).head(10)
fig4 = go.Figure()
fig4.add_trace(go.Bar(x=df_att["Squad"], y=df_att["AttendanceN"], text=df_att["Attendance"], textposition="outside"))
fig4.update_layout(title="Attendance (Top 10, filtered)", xaxis_title="", yaxis_title="Attendance", height=430, margin=dict(l=20,r=20,t=60,b=40))
fig4.show()

# 5.5 Last 5 (stacked W/D/L)
order = df_f.sort_values(["W5","D5","L5"], ascending=[False, False, True])
fig5 = go.Figure()
fig5.add_trace(go.Bar(name="W", x=order["Squad"], y=order["W5"]))
fig5.add_trace(go.Bar(name="D", x=order["Squad"], y=order["D5"]))
fig5.add_trace(go.Bar(name="L", x=order["Squad"], y=order["L5"]))
fig5.update_layout(barmode="stack", title="Last 5 Results (W/D/L, filtered)", yaxis_title="Matches", height=480, margin=dict(l=20,r=20,t=60,b=40))
fig5.show()

# 5.6 xG vs xGA (grouped)
order2 = df_f.sort_values("xG", ascending=False)
fig6 = go.Figure()
fig6.add_trace(go.Bar(name="xG",  x=order2["Squad"], y=order2["xG"]))
fig6.add_trace(go.Bar(name="xGA", x=order2["Squad"], y=order2["xGA"]))
fig6.update_layout(barmode="group", title="xG vs xGA (filtered)", yaxis_title="Expected Goals", height=480, margin=dict(l=20,r=20,t=60,b=40))
fig6.show()
